In [1]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
import base64
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps


# import CRUD Python module file 
from animal_crud import AnimalShelter





###########################
# Data Manipulation / Model
###########################
# username and password and CRUD Python module name
username = "aacuser1"
password = "Tastee05"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#Grazioso Salvare’s logo
image_filename = 'Grazioso_Salvare_Logo.png' 
encoded_image = base64.b64encode(open(image_filename, 'rb').read())
#creator image
image_filename = 'gunner_image.jpg' 
encoded_image_gunner = base64.b64encode(open(image_filename, 'rb').read())


app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.A(
        [
        
            html.Img(
            src = "data:image/png;base64,{}".format(
                encoded_image.decode()),
            style={"height": "2" "00px"},
            )
            
        ],
        href="https://www.snhu.edu",
    ),

            
    html.Img(
        src = "data:image/png;base64,{}".format(encoded_image_gunner.decode()),
        style={"height": "2" "00px"},
                
            ),
    html.Hr(),
    ##Radio items for user to select for interactive table and charts
    html.Div(
        dcc.RadioItems(
        id = 'rescue-dog',
        options = [
        {'label' : 'Water Rescue', 'value' : 'Water'},
        {'label' : 'Mountain or Wilderness Rescue', 'value' : 'Mountain'},
        {'label' : 'Disaster or Individual Tracking', 'value' : 'Disaster'},
        {'label' : 'Reset', 'value' : 'All'}
        ],
        value = 'All',
        ),
    ),
    html.Hr(),
        dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable = False,
        filter_action = "native",
        sort_mode = "multi",
        column_selectable = False, 
        row_selectable = "single", #Enables selected animal to show on map
        row_deletable = False,
        selected_columns = [],
        selected_rows = [],
        page_action = "native",
        page_current = 0,
        page_size = 10,

        
    ),
    html.Br(),
    html.Hr(),



#This sets up the dashboard so piechart and geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            dcc.Graph(id='graph-id'),
            className='col s12 m6',

            ),
        html.Div(
       
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################
  

@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

#Output radio items, changes with selections for rescue type or reset

@app.callback(
    Output('datatable-id', 'data'),
    
    [Input('rescue-dog', 'value')]
)

def build_table(value):
    if value == 'All':
        df = pd.DataFrame.from_records(shelter.read({}))
        return df.to_dict('records')
    elif value == 'Water':
        df = pd.DataFrame(list(shelter.read({"animal_type":"Dog","breed":{"$in":["Labrador Retriever Mix","Chesapeake Bay Retriever",
                                            "Newfoundland"]},"sex_upon_outcome":"Intact Female", "age_upon_outcome_in_weeks":{"$gte":26},
                                             "age_upon_outcome_in_weeks":{"$lte":156}})))
        return df.to_dict('records')
    elif value == 'Mountain':
        df = pd.DataFrame(list(shelter.read({"animal_type":"Dog","breed":{"$in":["German Shepherd","Alaskan Malamute",
                                            "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},"sex_upon_outcome":"Intact Male", "age_upon_outcome_in_weeks":{"$gte":26},
                                             "age_upon_outcome_in_weeks":{"$lte":156}})))
        return df.to_dict('records')
    elif value == 'Disaster':
        df = pd.DataFrame(list(shelter.read({"animal_type":"Dog","breed":{"$in":["Doberman Pinscher","German Shepherd",
                                            "Golden Retriever", "Bloodhound", "Rottweiler"]},"sex_upon_outcome":"Intact Male", "age_upon_outcome_in_weeks":{"$gte":20},
                                             "age_upon_outcome_in_weeks":{"$lte":300}})))
        return df.to_dict('records')



#Populate and update Pie chart with radio selections
@app.callback(
    Output('graph-id', 'figure'),
    [Input('rescue-dog', 'value')
    ]
    
    )
def update_graphs(value):
    if value == 'All':
        dff = df
        fig = px.pie(
        dff, names = 'breed',
        )    
    
        return (fig)  
    elif value == 'Water':
        dff = pd.DataFrame(list(shelter.read({"animal_type":"Dog","breed":{"$in":["Labrador Retriever Mix","Chesapeake Bay Retriever",
                                            "Newfoundland"]},"sex_upon_outcome":"Intact Female", "age_upon_outcome_in_weeks":{"$gte":26},
                                             "age_upon_outcome_in_weeks":{"$lte":156}})))
        fig = px.pie(
        dff, names = 'breed',
        )    
    
        return (fig)  
    elif value == 'Mountain':
        dff = pd.DataFrame(list(shelter.read({"animal_type":"Dog","breed":{"$in":["German Shepherd","Alaskan Malamute",
                                            "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},"sex_upon_outcome":"Intact Male", "age_upon_outcome_in_weeks":{"$gte":26},
                                             "age_upon_outcome_in_weeks":{"$lte":156}})))
        fig = px.pie(
        dff, names = 'breed',
        )    
    
        return (fig)  
    elif value == 'Disaster':
        dff = pd.DataFrame(list(shelter.read({"animal_type":"Dog","breed":{"$in":["Doberman Pinscher","German Shepherd",
                                            "Golden Retriever", "Bloodhound", "Rottweiler"]},"sex_upon_outcome":"Intact Male", "age_upon_outcome_in_weeks":{"$gte":20},
                                             "age_upon_outcome_in_weeks":{"$lte":300}})))  
        fig = px.pie(
        dff, names = 'breed',
        )    
        return (fig)

    

        fig = px.pie(
        dff, names = 'breed',
        )    
    
        return (fig)   
    


#Populate and update map with radio and row selections
@app.callback(
    Output('map-id', "children"),
    [
        Input('datatable-id', "derived_virtual_data"),
        Input('datatable-id', "derived_virtual_selected_rows"),
    ])


   
def update_map(derived_virtual_data, selected_row_index): 

    dff = df if selected_row_index is None else pd.DataFrame(derived_virtual_data)
    if selected_row_index is None or len(selected_row_index) == 0:
        return [
            dl.Map(style = {'width': '1000px', 'height' : '500px'}, center=[30.75,-97.48], zoom=10, children = [
                dl.TileLayer(id = "base-layer-id"),
            #Marker with tool tip and popup
                dl.Marker(position = [30.75,-97.48], children=[
                    dl.Tooltip(dff.iloc[0,4]),
                    dl.Popup([
                        html.H1("Animal Name"),
                        html.P(dff.iloc[0,9])#changed from 1 to 0
                    ])
                ])
            ])
        ]
    else:
        return [

            dl.Map(style = {'width': '1000px', 'height' : '500px'}, 
                center=[
                    float(dff.iloc[selected_row_index, 13].values[0]), 
                    float(dff.iloc[selected_row_index, 14].values[0]), 
                    
                    ], 
                zoom=10, 
                children = [
                dl.TileLayer(id = "base-layer-id"),
                #Marker with tool tip and popup
                dl.Marker(
                    position = [
                    float(dff.iloc[selected_row_index, 13].values[0]), 
                    float(dff.iloc[selected_row_index, 14].values[0]),  
                    ],
                      
                     
                children=[
                    dl.Tooltip(dff.iloc[selected_row_index,4]),
                    dl.Popup([
                        html.H1("Animal Name"), 
                        html.P(dff.iloc[selected_row_index,9])
                            ])
                        ])
        
    ])
    ]
        




app